In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import numpy as np
import math

/home/witvoetk/anaconda3/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = TabularDataset(f'Data/phil_rangedata_train.csv')
train_data.head()

,initial_SOC,final_SOC,altitude,avg_speed,total_distance
0,100.000031,100.000031,-3.061713,7.642350,4585.410159
1,100.000031,100.000031,-0.397832,4.726447,2822.495564
2,100.000031,99.607872,-2.267668,5.793851,3476.310818
3,100.000031,99.215721,-10.155657,11.395269,6837.161613
4,99.607872,97.254936,-10.348317,19.169618,11486.297899


In [3]:
label = 'final_SOC'
train_data[label].describe()

KeyError: 'Final SOC'

In [ ]:
predictor = TabularPredictor(label=label,eval_metric='root_mean_squared_error').fit(
    train_data, 
    presets=['best_quality','optimize_for_deployment'], 
    time_limit = 30*60,
    verbosity=1) 

In [ ]:
test_data = TabularDataset(f'Data/phil_rangedata_test.csv')

y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

In [ ]:
predictor.evaluate(test_data, silent=True)

In [ ]:
predictor.leaderboard(test_data, silent=True)

In [ ]:
#plots the predicted vs actual values of the top performing model using matplotlib
import matplotlib.pyplot as plt
plt.plot(y_pred, label="Predictions")
plt.plot(test_data[label], label="True Values")
plt.legend()
plt.show()

In [ ]:
mse_test = np.mean(((y_pred - test_data[label])**2))
rmse_test = math.sqrt(mse_test)
print("test data rmse", rmse_test)